In [ ]:
import pandas as pd
import numpy as np,re
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

# Авторизация в сервисе GigaChat
chat = GigaChat(credentials='MzU0M2RmZmMtZTZiYS00NDY2LWE1ODQtZTUzMTFhNTBjZmQ0OjcxYWE1NGY1LWUxZTctNDk1NC1hZGViLTkwOTIyODNiOWVhYw==', verify_ssl_certs=False)

messages = [
    SystemMessage(
        content="Ты лучший аналитик в компании РЖД, необходмо проанализировать данные респондента и вывести вероятность того, уйдет ли респондент от компании и перестанет пользоваться их услугами. Ты даёшь свой ответ только в виде % ухода пользователя и обоснование поставленного процента"
    )
]

In [75]:
def parse_xls(filename: str):
    file = pd.read_excel(filename)
    rows = file.loc
    lst = []
    for row in file.itertuples(index=False):
        rw = []
        for r in row:
            rw.append(r)
        lst.append(rw)
    return lst

In [5]:
waitings = parse_xls("Интересы.xls")
zayavki = parse_xls("Обращения.xls")
obama = parse_xls("Объёмы перевозок.xls")
all_data = [waitings, zayavki, obama]

In [6]:
id = int(input("Введите ID:"))
for i in all_data:
    for k in i:
        for z in k:
            if z == id:
                print(k)
                break
            


['15.04.2024 10:24', 'Привлечение клиента на новое направление Стальная- Караганда-Угольная(041)', '6. Продажа услуг действующим клиентам', 'Завершен успешно', 912, 'ЦПУ КИРОВ', 10.0, 100.0, nan, nan, '10. Холодный звонок', '0001-097217', 'Интерес "Привлечение клиента на новое направление Стальная- Караганда-Угольная(041)" от 15.04.2024, 10 руб.']
['20.03.2024 14:31', 'Привлечение клиента на новое направление Стальная - Вторая Речка(041)', '6. Продажа услуг действующим клиентам', 'Завершен успешно', 912, 'ЦПУ КИРОВ', 10.0, 100.0, nan, nan, '10. Холодный звонок', '0001-074580', 'Интерес "Привлечение клиента на новое направление Стальная - Вторая Речка(041)" от 20.03.2024, 10 руб.']
['24.06.2024 16:37', 'Услуга Аренда локомотива ОАО "РЖД"', '5. Продажа услуг аренды локомотивов', 'Завершен неудачно', 912, 'ЦПУ МУРАШИ', nan, nan, nan, nan, '10. Холодный звонок', '0001-384232', 'Интерес "Услуга Аренда локомотива ОАО "РЖД"" от 24.06.2024']
['26.03.2024 15:11', 'Привлечение клиента на новое н

In [ ]:
               Дополнительная информация которая тебе может помочь: {personal_dop_info}

In [7]:
# Вводим ID Получаем  -> list ["ID", "%", "Вывод", "Рекомендации"]

class ML_AGEENT:
    def __init__(self) -> None:
        self.dataframe_interesting = pd.read_excel("Интересы.xls")
        self.dataframe_waitings = pd.read_excel("Обращения.xls")
        self.dataframe_transporting = pd.read_excel("Сортированный_Объем Перевозок.xlsx")
        self.dataframe_Vladimyr = pd.read_excel("МС_Владимирская область.xls")
        self.dataframe_Kirov = pd.read_excel("МС_Кировская область.xls")
        self.dataframe_NNov = pd.read_excel("МС_Нижегородская область.xls")
        self.dataframe_MaryEl = pd.read_excel("МС_Республика Марий Эл.xls")
        self.dataframe_Mordoviya = pd.read_excel("МС_Республика Мордовия.xls")
        self.dataframe_Tatarstan = pd.read_excel("МС_Республика Татарстан.xls")
        self.dataframe_Ydmurt = pd.read_excel("МС_Республика Удмуртия.xls")
        self.dataframe_Chyvash = pd.read_excel("МС_Республика Чувашия.xls")
        self.oblast_list = [
                            self.dataframe_Vladimyr, 
                            self.dataframe_Kirov, 
                            self.dataframe_NNov, 
                            self.dataframe_MaryEl, 
                            self.dataframe_Mordoviya, 
                            self.dataframe_Tatarstan,
                            self.dataframe_Ydmurt,
                            self.dataframe_Chyvash
                            ]


    def research(self, id:str) -> list:
        
        _, personal_data = self.research_oblasts(id)
        _, personal_seasons = self.seasons(id)
        _, personal_dop_info,sum = self.research_directions(id)
        persnal_problems = self.research_obrash(id)
        persnal_inters = self.research_interesting(id)
        
        prompt = f"""У нас есть информация о клиенте железнодорожной компании и ключевое значение для анализа:

Значение {sum} - общая сумма платежей клиента компании за последние 9 месяцев. Важно отметить то, что:

Если {sum} > 0, проанализируйте предоставленные данные и оцените вероятность ухода.
Если {sum} == 0, вероятность ухода составляет 100%, и следует предоставить рекомендации для возврата клиента.

Данные о клиенте:

Информация о юридическом лице клиента, платежеспособности и рисках: {personal_data}
Данные о грузоперевозках клиента за последние два года: {personal_seasons}
Информация о взаимодействиях клиента с компанией, таких как обращения за консультированием, поддержка, жалобы, благодарности и т.д.: {persnal_inters}
Данные о проблемах клиента, возникших ранее: {persnal_problems}
Пожалуйста, на основе этих данных, если {sum} > 0, оцените вероятность ухода клиента и предоставьте рекомендации для удержания клиента, если вероятность ухода составляет более 45%. Если {sum} == 0, предложите действия для возвращения клиента с высокой вероятностью ухода (100%)

Сами же данные нужно представить в виде: 

Вероятность ухода: <оценка вероятности ухода в процентах>
Обоснование: <основной текст>"""
        messages.append(HumanMessage(content=prompt))
        res = chat(messages)
        messages.append(res)
        
        pattern = r"Вероятность ухода:\s*(\d+%)\nОбоснование:\s*(.*)"
        
        match = re.search(pattern, res.content, re.DOTALL)

        if match:
            # Извлечение данных из групп регулярного выражения
            churn_probability = match.group(1)  # Вероятность ухода
            justification = match.group(2)  # Обоснование
            
            # Формируем структурированный ответ
            structured_data = {
                "churn_probability": churn_probability,
                "justification": justification
            }
            
            return structured_data
        
        ...
    
    def research_oblasts(self, id:str) -> str:
        for i in self.oblast_list:
            filtred = i[i["ID"] == id][["ID",
                                        "Размер компании.Наименование", 
                                        "Карточка клиента (внешний источник).Индекс платежной дисциплины Описание", 
                                        "Карточка клиента (внешний источник).Индекс финансового риска Описание", 
                                        "Госконтракты.Тип контракта"]]
            if len(filtred.values.tolist()) >=1:
                break
            else:
                pass
        
        all_data = filtred.values.T.tolist()
        # print(all_data[0], all_data[1])
        all_data = [list(set(sublist)) for sublist in all_data]
        # print(all_data[0], all_data[1])
        
        final = "Этот клиент - " +   str(all_data[1]) + " C " + str(all_data[2]) + " С индексом финального риска: " + str(all_data[3]) + " И в гос контрактах он является: " + str(all_data[4])
        # print(final)
        return filtred, final
    
    def seasons(self, id:str) ->  str:
        
        filtred = self.dataframe_transporting[self.dataframe_transporting["ID"] == id]
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        month_normal = years[0::2]
        month_in_numbers =  [[sum(subarray)] for subarray in month]
        month_in_tons = month_in_numbers[1::2]
        month_in_money = month_in_numbers[0::2]
        text = ""
        for i in range(len(month_normal)):
            text += f"Клиент в год, месяц - {month_normal[i]} перевёз {month_in_tons[i]} тон груза  на сумму: {month_in_money[i]}\n"
            
        # print(text)
        
        return filtred, text

    def research_directions(self, id:str):
        
        filtred=self.dataframe_transporting[self.dataframe_transporting['ID']==id]
        all_data = filtred.values.T.tolist()
        
        text=""
        sum=0
        
        for i in range(len(all_data[3])):

            text+="| Этот клиент везет из - " + all_data[1][i] + " в - " + all_data[2][i] + "| Код товара - " + str(all_data[3][i])+"| Категория груза - "+all_data[4][i]+"|\n"
            
        # print (text)
        
        years = filtred.columns.values.tolist()[5:]
        month = filtred[years].values.T.tolist()
        
        
        for i in month[-8::2]:
            
            for j in i:
                
                sum+=j
        
        return all_data, text,sum

    def research_interesting(self, id:str) -> str:
        filtred = self.dataframe_interesting[self.dataframe_interesting["ID"] == id][["Состояние", "Канал первичного интереса"]]

        all_data = filtred.values.T.tolist()
        all_data = [list(set(sublist)) for sublist in all_data]

        string = ""
        ls = []

        for i in filtred.values:
            string += str(i[0]) + " " + str(i[1]) + ", "

        for i in all_data[0]:
            for j in all_data[1]:
                if string.count(f"{i} {j}") != 0:
                    ls.append([str(i) + " " + str(j), string.count(f"{i} {j}")])

        for i in ls:
            string += i[0] + " " + str(i[1]) + "\n"

        return string
    

    def research_obrash(self, id:str) -> str:
        filtred = self.dataframe_waitings[self.dataframe_waitings["ID"] == id][["Тема вопроса"]]

        all_data = filtred.values.T.tolist()
        try:
            
            string = ""
            for i in all_data:
                string += i[0] + "\n"

            print(string)
        except:
            string = "Обращений не обнаружено"
        
        return string
 
    def graps_transp(self, id:str) -> list:
        filtred = self.dataframe_transporting[self.dataframe_transporting["ID"] == id]
        years = filtred.columns.values.tolist()[5:][0::2]
        types = list(set(filtred["Гр груза по опер.номен"]))
        tat = []
        for i in types:
            temp = filtred[filtred["Гр груза по опер.номен"] == i]
            data = temp[years].values.T.tolist()
            tat.append([[sum(subarray)] for subarray in data])
            
            
            
        return tat, years, types
        
        
    

In [8]:
agent = ML_AGEENT()

In [78]:
tat, years, types = agent.graps_transp(912)


In [116]:
pipa = []
for i in range(len(tat)):
    tata = []
    for k in range(len(tat[i])):
        tata.append(tat[i][k][0])
    pipa.append(tata)   
pipa[1]

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 26792,
 26792,
 187544,
 53584,
 53673,
 112223,
 0,
 0,
 70540,
 35270,
 153300,
 192478,
 148244,
 157208,
 157208]

In [117]:
import plotly.graph_objects as go
animals=years

fig = go.Figure()
for i in range(len(types)):
    fig.add_trace(go.Bar(name=types[i], x=years, y=pipa[i]),)
fig.update_layout(barmode='group')
fig.show()


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Загрузка модели и токенизатора
model_name = "EleutherAI/gpt-neo-1.3B"  # или "EleutherAI/gpt-neo-1.3B" для более мощной модели
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Генерация текста
input_text = "Каковы перспективы развития искусственного интеллекта?"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Генерация ответа
output = model.generate(input_ids, max_length=100, num_return_sequences=1)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

c:\Users\User\Desktop\project\Vector_2024_rzhd\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--EleutherAI--gpt-neo-1.3B. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using a model of type gpt_neo to instantiate a model of type gpt2. This is not suppor

KeyboardInterrupt: 

In [4]:
import spacy

# Загрузка предобученной модели
nlp = spacy.load("en_core_web_sm")

# Обработка текста
text = "Apple is looking at buying U.K. startup for $1 billion"
doc = nlp(text)

# Извлечение сущностей
for ent in doc.ents:
    print(ent.text, ent.label_)


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [5]:
from langchain import LLMChain
from langchain.llms import OpenAI

# Инициализация модели
llm = OpenAI(model_name="gpt-3.5-turbo")

# Создание цепочки
chain = LLMChain(llm=llm)

# Генерация текста
response = chain.run("Каковы перспективы развития искусственного интеллекта?")
print(response)

c:\Users\User\Desktop\project\Vector_2024_rzhd\.venv\Lib\site-packages\langchain_community\llms\openai.py:254: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


ValidationError: 1 validation error for OpenAIChat
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [7]:

from transformers import pipeline

# Использование BERT для извлечения сущностей
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
text = "Apple is looking at buying U.K. startup for $1 billion"
entities = ner_pipeline(text)
print(entities)

c:\Users\User\Desktop\project\Vector_2024_rzhd\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--dbmdz--bert-large-cased-finetuned-conll03-english. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at dbmdz/bert-large-cased-finet

[{'entity': 'I-ORG', 'score': 0.9990183, 'index': 1, 'word': 'Apple', 'start': 0, 'end': 5}, {'entity': 'I-LOC', 'score': 0.9996722, 'index': 6, 'word': 'U', 'start': 27, 'end': 28}, {'entity': 'I-LOC', 'score': 0.9979365, 'index': 8, 'word': 'K', 'start': 29, 'end': 30}]


In [8]:
text_generator = pipeline("text-generation", model="gpt2")
generated_text = text_generator("Once upon a time", max_length=50)
print(generated_text)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': "Once upon a time I worked at the London Underground depot I didn't know it was for a reason? To be honest, the idea of getting something back from an old friend was so exciting that I asked for it. I did this in order to"}]


In [19]:
import requests

# Задаем адрес API
url = "http://127.0.0.1:5500"

# Задаем параметры запроса
params = {
    "text": "Напиши привет мир на питоне"  # Замените на ваш вопрос
}

# Отправляем GET-запрос
response = requests.get(url, params=params)

print(response)
# Проверяем статус код ответа
if response.status_code == 200:
    # Если запрос успешен, выводим ответ
    print("Ответ от API:", response.text)
else:
    # Если произошла ошибка, выводим статус код
    print("Ошибка:", response.status_code)

<Response [200]>
Ответ от API: print("Привет, мир!")


In [ ]:
from ctransformers import AutoModelForCausalLM

# Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GGUF", model_file="mistral-7b-openorca.Q4_0.gguf", model_type="mistral", gpu_layers=0)

print(llm("AI is going to"))

c:\Users\User\Desktop\project\Vector_2024_rzhd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

'10 years ago, I was in the process of moving to New York City. It was an exciting and scary time for me as it meant leaving my family and friends behind to start fresh in this big city.\n'